In [1]:
import numpy as np


def move_is_correct(grid,num):
    '''
    @param grid: 6x7 grid containing the current game state
    @param num: column

    returns True if move is allowed on that column
    '''

    #if 0 is in column
    if 0 in grid[:,num]:
    
        #move is allowed
        return True

    else:

        return False

def move_still_possible(S):
    '''
    @param S: 6x7 grid containing the current game state
    returns True if grid contains no 0, therefore no move possible anymore
    '''
    return not(S[S==0].size == 0)


def move(S,p,col_num):
    '''
    @param S: 6x7 grid containing the current game state
    @param p: current player
    @param col_num: column number
    
    sets the player's number on the grid and returns the grid
    '''
    
    #sanity check
    if 0 in S[:,col_num]:    
        y = np.where(S[:,col_num]==0)[0][-1]
        S[y,col_num] = p
        return S , y, col_num
    else:
        return S, None, None
        return 

def move_at_random(S):
    '''
    @param S: 6x7 grid containing the current game state
    moves at random
    '''
    return np.random.randint(0,S.shape[1])


#neat and ugly but the fastest way to search a matrix for a vector is a string find
player1=[_,_, '1 1', '1 1 1', '1 1 1 1']
oponent=[_,_, '2 2', '2 2 2', '2 2 2 2']

#good vectors with high winning probability
player1prob=['0 0 0 1 0 0 0','0 0 1 0 0','0 0 0 1','1 0 0 0',' 0 0 1 0','0 1 0 0','0 1 1 0',' 1 0 0', '0 0 1']


def move_was_winning_move(S, p, num):
    '''
    @param S: 6x7 grid containing the current game state
    @param p: current player
    @param num: how many occurences to count
    
    combines all the allowed formations of the grid and string_finds with 
    the currents player vector. Returns true if match.
    
    return: True or False whether move was winning move or not,
            and count of occurences
    '''
    if p == 1:
        match = player1[num]
    else:
        match = oponent[num]

    l=[]
    #for every possible diag
    for i in range(-2,4):
        l.append(np.diag(S,k = i))
        l.append(np.diag(np.fliplr(S),k=i))
    #left to right
    l.append(S)
    #top to bottom
    l.append(np.rot90(S)) 
    
    #convert to string
    stringmatrix =''.join(np.array_str(e) for e in l)
    
    #count the occurences
    counter = stringmatrix.count(match)
    
    #print stringmatrix
    
    #if four in a row
    if num == 4 and counter == 1:
        return True, counter
    return False, counter





# relate numbers (1, -1, 0) to symbols ('x', 'o', ' ')
symbols = {1:'b', 2:'r', 0:' '}

# print game state matrix using symbols
def print_game_state(S):
    B = np.copy(S).astype(object)
    for n in [1, 2, 0]:
        B[B==n] = symbols[n]
    print B





if __name__ == '__main__':
    # initialize 6x7 connectfour board
    gameState = np.zeros((6,7), dtype=int)

    # initialize player number, move counter
    player = 1
    mvcntr = 1

    # initialize flag that indicates win
    noWinnerYet = True
    

    while move_still_possible(gameState) and noWinnerYet:
        
        while True:
            # get player symbol
            name = symbols[player]
            print '%s moves' % name

            # let player move at random
            col_num = move_at_random(gameState)
            if move_is_correct(gameState, col_num):
                gameState, _ , _ = move(gameState,player,col_num)

                # print current game state
                print_game_state(gameState)

                # evaluate game state
                winningmove, _ = move_was_winning_move(gameState, player, 4)
                if winningmove:
                    print 'player %s wins after %d moves' % (name, mvcntr)
                    noWinnerYet = False
                

                # switch player and increase move counter
                if player == 1:
                    player = 2
                elif player == 2:
                    player = 1

                mvcntr +=  1
                
                break




    if noWinnerYet:
        print 'game ended in a draw' 




b moves
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'b' ' ' ' ' ' ' ' ' ' ']]
r moves
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' 'b' 'r' ' ' ' ' ' ' ' ']]
b moves
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['b' 'b' 'r' ' ' ' ' ' ' ' ']]
r moves
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'r' ' ' ' ' ' ' ' ']
 ['b' 'b' 'r' ' ' ' ' ' ' ' ']]
b moves
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'r' ' ' ' ' ' ' ' ']
 ['b' 'b' 'r' ' ' 'b' ' ' ' ']]
r moves
[[' ' ' ' ' ' ' '

In [28]:
class Minmax(object):

           
    #heuristic to evaluate the next best move
    def score(self, state, cur_player):
        '''
        state: current gameState
        cur_player: current player
        
        we are using a simple heuristic here,
        just counting and punishing good and reward good moves with a weight:
        
        score = num of(four in row)*1000+ num of(three in row)* 100+ num of(two in row)*10
        - num of opponent(four in row)*100000- num of opponent(three in row)*100- 
        num of opponent(two in row)*10
        
        -> todo:+ awarding if left and right of coin min 3 emtpy
        
        returns the score
        '''
        
        if cur_player == 1:
            oponent = 2
        else:
            oponent = 1
        
        #-> improvement here: give a list and call move_was_winning_move once
        
        #counting the number of good four/threes/twos in a row/column/diag
        #_ , b = zip(*(move_was_winning_move(gameState, cur_player, e) for e in range(2,5)))
        #_ , c = zip(*(move_was_winning_move(gameState, oponent, f) for f in range(2,5)))


        
        _, fours_player = move_was_winning_move(state, cur_player, 4)
        _, threes_player = move_was_winning_move(state, cur_player, 3)
        _, twos_player = move_was_winning_move(state, cur_player, 2)
        
        _, fours_oponent = move_was_winning_move(state, oponent, 4)
        _, threes_oponent = move_was_winning_move(state, oponent, 3)
        _, twos_oponent = move_was_winning_move(state, oponent, 2)
        
        #score = b[0]*10+b[1]*100+b[2]*1000-c[0]*10-c[1]*100-c[2]*100000
        #score2 = b[2]*1000+b[1]*100+b[0]*10-c[2]*100000-c[1]*100-c[0]*10
        score = fours_player*1000+threes_player*100+twos_player*10-fours_oponent*100000-threes_oponent*100-twos_oponent*10
        #print 'score2', score2
        return score
    
    def listofmoves(self, gameState):
        '''
        returns a list of possible moves = column not full
        
        gameState: current gamestate
        
        return: list of possible moves
        '''
        
        l=[]
        for i in range(gameState.shape[1]):
            if 0 in gameState[:,i]:
                l.append(i)
        return l
        
    
    def min_play(self, board, depth,alpha ,beta , cur_player):
        '''
        recursively building a the tree, min part of minmax
        minimzing the score, moving the oponent
        
        board: a gameState 
        depth: depth parameter of search tree
        cur_player: the current_player's number
        
        return best score
        '''
        print board,'depth: ', depth


        #termination
        #max depth
        if depth == 0:
            return self.score(board, cur_player)
        #all full
        elif not move_still_possible(board):
            return self.score(board, cur_player)
        #or winner
        winningmove, _ = move_was_winning_move(board, cur_player, 4)
        if winningmove:
            return self.score(board, cur_player)
        
        #eval current player
        if cur_player == 1:
            oponent = 2
        else:
            oponent = 1
        
        best_score = np.inf
        
        #get all available moves
        moves = self.listofmoves(board)
        best_move = moves[0]
        
        if len(moves) == 0:
            return self.score(board, cur_player)
        
        for eachmove in moves:
            #print 'min', eachmove
            #copy board and move oponent
            boardcopy = board.copy()
            
            board_i, _ , _ = move(boardcopy, oponent, eachmove)
            
            #build recursivley max
            score = self.max_play(board_i, depth-1,alpha ,beta , cur_player)
                        
            #compare scores MIN
            #if score < best_score:
            #    best_move = eachmove
            #    best_score = score
                #print 'if', best_move, best_score
            if score < beta:
                beta = score
                #best_move = eachmove
            if beta <= alpha:
                return beta

        
        return beta#, best_move
    
    
    def max_play(self, board, depth,alpha ,beta , cur_player):
        '''
        recursively building a the tree, max part of minmax
        maximizing the score
        
        board: a gameState 
        depth: depth parameter of search tree
        cur_player: the current_player's number
        
        return best score
        '''
        print board,'depth: ', depth


        #termination
        #max depth
        if depth == 0:
            return self.score(board, cur_player)
        #all full
        elif not move_still_possible(board):
            return self.score(board, cur_player)
        #or winner
        winningmove, _ = move_was_winning_move(board, cur_player, 4)
        if winningmove:
            return self.score(board, cur_player)
        
        #eval current player
        if cur_player == 1:
            oponent = 2
        else:
            oponent = 1
        
        best_score = -np.inf
        
        #get all available moves
        moves = self.listofmoves(board)
        best_move = moves[0]
        
        if len(moves) == 0:
            return self.score(board, cur_player)
        
        for eachmove in moves:
            #print 'max', eachmove

            #copy board and move player
            boardcopy = board.copy()
            
            board_i, _ , _ = move(boardcopy, cur_player, eachmove)
            
            #build recursivley min
            score = self.min_play(board_i, depth-1,alpha ,beta , cur_player)
                        
            #compare scores MAX
            #if score > best_score:
            #    best_move = eachmove
            #    best_score = score
                #print 'if', best_move, best_score

            if score > alpha:
                alpha = score
                #best_move = eachmove
            if alpha >= beta:
                return alpha
        
        return alpha #, best_move
    
    
    def minmax(self, board, depth, alpha, beta, cur_player):
        '''
        recursively building a the tree with alpha beta pruning
        may not be the best choice but everyone keeps saying: memory is cheap
        
        board: a gameState 
        depth: depth parameter of search tree
        cur_player: the current_player's number
        
        return best score, best move
        '''
        
        print board,'depth: ', depth
        
        #eval current player
        if cur_player == 1:
            oponent = 2
        else:
            oponent = 1
        
        best_score = -np.inf
        
        #get all available moves
        moves = self.listofmoves(board)
        best_move = moves[0]

        #for each move do
        for eachmove in moves:
            #print eachmove
            #copy board and move
            boardcopy = board.copy()
            
            #build recursivley
            board_i, _ , _ = move(boardcopy, cur_player, eachmove)
            
            score = self.min_play(board_i, depth-1 ,alpha ,beta,  cur_player)
                        
            #compare scores
            if score > alpha:
                alpha = score
                best_move = eachmove
            if alpha >= beta:
                return alpha
            
            #if score > best_score:
             #   best_move = eachmove
             #   best_score = score
                
        #print 'if move', best_move,'score:', best_score  
        return alpha, best_move

In [29]:
gameState= np.array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 2, 1, 0],
       [0, 0, 0, 2, 2, 2, 0],
       [0, 0, 0, 1, 1, 1, 0]])

In [30]:
m = Minmax()

In [40]:
m.minmax(gameState, 5, -np.inf, np.inf, 1)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0]
 [0 0 0 0 2 1 0]
 [0 0 0 2 2 2 0]
 [0 0 0 1 1 1 0]] depth:  5
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0]
 [0 0 0 0 2 1 0]
 [0 0 0 2 2 2 0]
 [1 0 0 1 1 1 0]] depth:  4
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0]
 [0 0 0 0 2 1 0]
 [2 0 0 2 2 2 0]
 [1 0 0 1 1 1 0]] depth:  3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0]
 [1 0 0 0 2 1 0]
 [2 0 0 2 2 2 0]
 [1 0 0 1 1 1 0]] depth:  2
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 0 0 0 2 0 0]
 [1 0 0 0 2 1 0]
 [2 0 0 2 2 2 0]
 [1 0 0 1 1 1 0]] depth:  1
[[0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [2 0 0 0 2 0 0]
 [1 0 0 0 2 1 0]
 [2 0 0 2 2 2 0]
 [1 0 0 1 1 1 0]] depth:  0
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 0 0 0 2 0 0]
 [1 0 0 0 2 1 0]
 [2 0 0 2 2 2 0]
 [1 1 0 1 1 1 0]] depth:  0
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 0 0 0 2 0 0]
 [1 0 0 0 2 1 0]
 [2 0 0 2 2 2 0]
 [1 0 1 1 1 1 0]] depth:  0
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 0 0 0 2 0 0]
 [1 0 0 1 2 1 0]
 [2 0 0 2 2 2 0]
 [1 0 0 1 1

(-99130, 5)

In [ ]:
todo. ALPHA BETA PRUNING

In [23]:
move_was_winning_move(gameState,1,3)

(False, 0)

In [ ]:
In order to achieve this we will subtract the depth, that is the number of turns,
or recursions, from the end game score, the more turns the lower the score, 
the fewer turns the higher the score. Updating our code from above we have something 
that looks like this:
    